In [1]:
import tensorflow as tf
import numpy as np

# Step 1: Create the Full Model
# This model will contain two layers, which will be split later
def create_full_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(40,)),  # Input layer with 40 units
        tf.keras.layers.Dense(50, activation='relu', name='layer_1'),  # First dense layer (Server)
        tf.keras.layers.Dense(20, activation='softmax', name='layer_2')  # Second dense layer (Client)
    ])
    return model

# Step 2: Create the Layer 1 Model (Server)
# This model only contains the first layer, producing an output of 50 units
def create_layer_1_model():
    layer_1_input = tf.keras.Input(shape=(40,))
    layer_1_output = tf.keras.layers.Dense(50, activation='relu', name='layer_1')(layer_1_input)
    return tf.keras.Model(inputs=layer_1_input, outputs=layer_1_output)

# Step 3: Create the Layer 2 Model (Client)
# This model only accepts the output of the first layer and produces the final inference
def create_layer_2_model():
    layer_2_input = tf.keras.Input(shape=(50,))
    layer_2_output = tf.keras.layers.Dense(20, activation='softmax', name='layer_2')(layer_2_input)
    return tf.keras.Model(inputs=layer_2_input, outputs=layer_2_output)

# Step 4: Save the Full Model for Verification
full_model = create_full_model()
full_model.summary()

# Step 5: Convert and Save the Full Model for Reference
full_converter = tf.lite.TFLiteConverter.from_keras_model(full_model)
full_tflite_model = full_converter.convert()
with open("full_large_model.tflite", "wb") as f:
    f.write(full_tflite_model)
print("Full model saved as 'full_large_model.tflite'.")

# Step 6: Create, Convert, and Save Layer 1 Model (Server)
layer_1_model = create_layer_1_model()
layer_1_converter = tf.lite.TFLiteConverter.from_keras_model(layer_1_model)
layer_1_tflite_model = layer_1_converter.convert()
with open("layer_1.tflite", "wb") as f:
    f.write(layer_1_tflite_model)
print("Layer 1 model saved as 'layer_1.tflite'.")

# Step 7: Create, Convert, and Save Layer 2 Model (Client)
layer_2_model = create_layer_2_model()
layer_2_converter = tf.lite.TFLiteConverter.from_keras_model(layer_2_model)
layer_2_tflite_model = layer_2_converter.convert()
with open("layer_2.tflite", "wb") as f:
    f.write(layer_2_tflite_model)
print("Layer 2 model saved as 'layer_2.tflite'.")



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer_1 (Dense)                 │ (None, 50)             │         2,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_2 (Dense)                 │ (None, 20)             │         1,020 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,070 (11.99 KB)

 Trainable params: 3,070 (11.99 KB)

 Non-trainable params: 0 (0.00 B)

...

Saved artifact at 'C:\Users\gxg_c\AppData\Local\Temp\tmpt3wu2apm'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 20), dtype=tf.float32, name=None)
Captures:
  2575034845840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2575034846016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2575035000080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2575034999904: TensorSpec(shape=(), dtype=tf.resource, name=None)
Full model saved as 'full_large_model.tflite'.
INFO:tensorflow:Assets written to: C:\Users\gxg_c\AppData\Local\Temp\tmp4izae_i5\assets


INFO:tensorflow:Assets written to: C:\Users\gxg_c\AppData\Local\Temp\tmp4izae_i5\assets


Saved artifact at 'C:\Users\gxg_c\AppData\Local\Temp\tmp4izae_i5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40), dtype=tf.float32, name='keras_tensor_3')
Output Type:
  TensorSpec(shape=(None, 50), dtype=tf.float32, name=None)
Captures:
  2575052829968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2575052831376: TensorSpec(shape=(), dtype=tf.resource, name=None)
Layer 1 model saved as 'layer_1.tflite'.
INFO:tensorflow:Assets written to: C:\Users\gxg_c\AppData\Local\Temp\tmpka799dck\assets


INFO:tensorflow:Assets written to: C:\Users\gxg_c\AppData\Local\Temp\tmpka799dck\assets


Saved artifact at 'C:\Users\gxg_c\AppData\Local\Temp\tmpka799dck'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50), dtype=tf.float32, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 20), dtype=tf.float32, name=None)
Captures:
  2575076246976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2575076245568: TensorSpec(shape=(), dtype=tf.resource, name=None)
Layer 2 model saved as 'layer_2.tflite'.


In [2]:
import tensorflow as tf
import numpy as np

# Step 1: Load the two separate models
layer_1_interpreter = tf.lite.Interpreter(model_path="layer_1.tflite")
layer_2_interpreter = tf.lite.Interpreter(model_path="layer_2.tflite")

# Step 2: Allocate tensors for each model
layer_1_interpreter.allocate_tensors()
layer_2_interpreter.allocate_tensors()

# Step 3: Define a fixed input for testing the models (shape: [1, 40])
fixed_input = np.array([[1.0] * 40], dtype=np.float32)

# Step 4: Set up the input and output for Layer 1
input_details_1 = layer_1_interpreter.get_input_details()
output_details_1 = layer_1_interpreter.get_output_details()

# Set the input tensor for Layer 1
layer_1_interpreter.set_tensor(input_details_1[0]['index'], fixed_input)

# Step 5: Run inference for Layer 1
layer_1_interpreter.invoke()

# Retrieve the output from Layer 1
intermediate_output = layer_1_interpreter.get_tensor(output_details_1[0]['index'])

# Step 6: Set up the input and output for Layer 2
input_details_2 = layer_2_interpreter.get_input_details()
output_details_2 = layer_2_interpreter.get_output_details()

# Set the output of Layer 1 as input for Layer 2
layer_2_interpreter.set_tensor(input_details_2[0]['index'], intermediate_output)

# Step 7: Run inference for Layer 2
layer_2_interpreter.invoke()

# Retrieve the final output from Layer 2
final_output = layer_2_interpreter.get_tensor(output_details_2[0]['index'])

# Step 8: Print the results for verification
print("Fixed Input to Layer 1:")
print(fixed_input)
print("\nOutput of Layer 1 (Intermediate Output):")
print(intermediate_output)
print("\nFinal Output from Layer 2:")
print(final_output)


Fixed Input to Layer 1:
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]

Output of Layer 1 (Intermediate Output):
[[0.5917076  0.         0.40699878 0.         0.09211442 0.
  0.         0.314151   0.8121104  0.         0.5349524  0.73000014
  0.         0.         0.         0.4383371  0.         0.6461518
  0.06840402 0.13271318 0.42011806 0.         0.14866412 0.45915344
  0.         0.         0.11904608 0.67657095 2.3576026  0.05960916
  0.37451148 1.4012756  0.         0.61700296 0.         0.
  0.         0.         0.14999902 0.         0.         0.29394466
  0.         0.         0.         1.0715884  0.         0.
  0.         0.30483097]]

Final Output from Layer 2:
[[0.02873782 0.04039312 0.08297987 0.04233496 0.04628451 0.0910559
  0.05405142 0.03008143 0.01676836 0.0731731  0.02388647 0.01823472
  0.05444761 0.02779255 0.09108072 0.03778155 0.02241213 0.0653901
  0.12386347 0.02925025]]
